In [25]:
import cv2
import os
import json
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [26]:
# --- KONFIGURASI ---
VIDEO_PATH = "C:/Users/andre/Documents/Binus/Lomba/compfest/vietnam10.mp4"
ZONE_JSONS = ["zona_enhanced.json", "zona_vietnam1.json"]
OUTPUT_DIR = "event_dataset_clipped"
TARGET_FRAMES = float('inf')
FRAME_INTERVAL = 30  # Ambil setiap 30 frame (1 detik jika 30 FPS)
MARGIN_RATIO = 0.3

In [27]:
# --- UTILITIES ---
def load_zones(json_paths, video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return []

    h, w = frame.shape[:2]
    polygons = []
    for path in json_paths:
        with open(path, 'r') as f:
            data = json.load(f)
            if isinstance(data, list):
                polygons.append(data)
            else:
                poly = data["polygon"]
                meta = data.get("video_metadata", {})
                ow, oh = meta.get("width", w), meta.get("height", h)
                scale_x, scale_y = w / ow, h / oh
                scaled = [[int(x * scale_x), int(y * scale_y)] for x, y in poly]
                polygons.append(scaled)
    return polygons

def point_in_polygon(x, y, polygon):
    return cv2.pointPolygonTest(np.array(polygon, np.int32), (int(x), int(y)), False) >= 0

def crop_with_margin(frame, box, margin_ratio=0.3):
    h_img, w_img = frame.shape[:2]
    x1, y1, x2, y2 = map(int, box)
    box_w = x2 - x1
    box_h = y2 - y1
    mx = int(box_w * margin_ratio)
    my = int(box_h * margin_ratio)
    x1_crop = max(0, x1 - mx)
    y1_crop = max(0, y1 - my)
    x2_crop = min(w_img, x2 + mx)
    y2_crop = min(h_img, y2 + my)
    return cv2.resize(frame[y1_crop:y2_crop, x1_crop:x2_crop], (224, 224))


In [29]:
# --- INISIALISASI ---
import time


model = YOLO('yolov8n.pt')
tracker = DeepSort(max_age=60, n_init=3)
red_zone_polygons = load_zones(ZONE_JSONS, VIDEO_PATH)

cap = cv2.VideoCapture(VIDEO_PATH)
frame_count = 0
saved_events = set()
vehicle_snapshots = defaultdict(list)
os.makedirs(OUTPUT_DIR, exist_ok=True)


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    results = model(frame, verbose=False)[0]
    detections = []
    for box in results.boxes:
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        if conf > 0.5 and model.names[cls_id] in ["car", "truck", "bus"]:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            w, h = x2 - x1, y2 - y1
            detections.append(([x1, y1, w, h], conf, cls_id))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = list(map(int, track.to_ltrb()))
        cx = (ltrb[0] + ltrb[2]) // 2
        cy = (ltrb[1] + ltrb[3]) // 2

        in_red_zone = any(point_in_polygon(cx, cy, zone) for zone in red_zone_polygons)

        if in_red_zone and frame_count % FRAME_INTERVAL == 0:
            if len(vehicle_snapshots[track_id]) < TARGET_FRAMES:
                vehicle_snapshots[track_id].append((frame.copy(), ltrb))
            

    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- SIMPAN FRAME ---
for track_id, snapshots in vehicle_snapshots.items():
    if len(snapshots) < 16:
        continue
    
    seq_dir = os.path.join(OUTPUT_DIR, f"vehicle_{int(track_id)}_{int(time.time())}")
    os.makedirs(seq_dir, exist_ok=True)

    for idx, (f, box) in enumerate(snapshots):
        cropped = crop_with_margin(f, box, margin_ratio=MARGIN_RATIO)
        cv2.imwrite(os.path.join(seq_dir, f"frame_{idx:03d}.jpg"), cropped)

    print(f"✅ Saved {len(snapshots)} cropped frames to {seq_dir}")

cap.release()
cv2.destroyAllWindows()


✅ Saved 19 cropped frames to event_dataset_clipped\vehicle_1_1754729372
